# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [30]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension
!pip install gmaps
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [31]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [32]:
# Read dataframe from csv created in WeatherPy 
city_data = pd.read_csv('output_data/export_city_df.csv')
city_data

,Country,City,Longitude,Latitude,Temperature(F),Humidity(%),Cloudiness(%),Wind_Speed(mph)
0,CD,Kikwit,18.82,-5.04,72.70,88,85,1.25
1,RU,Tarko-Sale,77.78,64.92,39.99,86,98,7.76
2,AR,Mar del Plata,-57.56,-38.00,64.40,59,50,8.05
3,PH,Basco,121.97,20.45,81.28,82,92,13.78
4,US,Charleston,-79.93,32.78,64.40,100,90,4.70
...,...,...,...,...,...,...,...,...
543,AU,Bundaberg,152.35,-24.85,75.20,50,75,13.87
544,IN,Āmli,73.02,20.28,73.11,85,31,3.27
545,GB,Edinburgh,-3.20,55.95,46.35,87,51,1.12
546,NZ,Te Anau,167.72,-45.42,43.09,81,39,2.55


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [33]:
# Store latitude and longitude in locations
locations = city_data[["Latitude", "Longitude"]].astype(float)

# Store Humidity and convert to float
humidity = city_data["Humidity(%)"].astype(float)
locations

,Latitude,Longitude
0,-5.04,18.82
1,64.92,77.78
2,-38.00,-57.56
3,20.45,121.97
4,32.78,-79.93
...,...,...
543,-24.85,152.35
544,20.28,73.02
545,55.95,-3.20
546,-45.42,167.72


In [34]:
# Plot Heatmap
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(humidity),
                                 point_radius = 4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [35]:
#Filter dataframe to find ideal weather conditions
ideal_city_data = city_data.loc[(city_data["Temperature(F)"] < 80) & 
                                (city_data["Temperature(F)"] > 70) &
                                (city_data["Wind_Speed(mph)"] < 10) &
                                (city_data["Cloudiness(%)"] == 0)]
ideal_city_data = ideal_city_data.reset_index(drop = True)   
ideal_city_data

,Country,City,Longitude,Latitude,Temperature(F),Humidity(%),Cloudiness(%),Wind_Speed(mph)
0,MZ,Angoche,39.91,-16.23,73.18,92,0,7.45
1,SA,Sakakah,40.21,29.97,78.80,21,0,8.05
2,SA,Riyadh,46.72,24.69,77.00,27,0,3.36
3,US,Hays,-98.03,30.05,72.68,41,0,1.48
4,IR,Sūsangerd,48.18,31.56,78.80,61,0,2.24
5,IN,Bilhaur,80.08,26.85,75.87,45,0,4.52
6,BR,Caravelas,-39.25,-17.71,74.50,84,0,7.49
7,SA,Buraidah,43.98,26.33,73.40,35,0,5.82
8,IL,Kiryat Gat,34.76,31.61,75.99,85,0,2.46
9,AR,San Juan,-68.54,-31.54,70.14,16,0,7.63


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [36]:
# set up additional columns to hold information
ideal_city_data["Hotel Name"] = ""
ideal_city_data

,Country,City,Longitude,Latitude,Temperature(F),Humidity(%),Cloudiness(%),Wind_Speed(mph),Hotel Name
0,MZ,Angoche,39.91,-16.23,73.18,92,0,7.45,
1,SA,Sakakah,40.21,29.97,78.80,21,0,8.05,
2,SA,Riyadh,46.72,24.69,77.00,27,0,3.36,
3,US,Hays,-98.03,30.05,72.68,41,0,1.48,
4,IR,Sūsangerd,48.18,31.56,78.80,61,0,2.24,
5,IN,Bilhaur,80.08,26.85,75.87,45,0,4.52,
6,BR,Caravelas,-39.25,-17.71,74.50,84,0,7.49,
7,SA,Buraidah,43.98,26.33,73.40,35,0,5.82,
8,IL,Kiryat Gat,34.76,31.61,75.99,85,0,2.46,
9,AR,San Juan,-68.54,-31.54,70.14,16,0,7.63,


In [37]:
# find the closest hotel of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "rankby": "distance",
    "type": "hotel",
    "key": gkey,
}
# use iterrows to iterate through pandas dataframe
for index, row in ideal_city_data.iterrows():

    # get longtitude and latitude from df
    long = row['Longitude']
    
    lat = row['Latitude']

    # add keyword to params dict
    params['location'] = f"{lat},{long}"

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        ideal_city_data.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        pass
    
ideal_city_data

,Country,City,Longitude,Latitude,Temperature(F),Humidity(%),Cloudiness(%),Wind_Speed(mph),Hotel Name
0,MZ,Angoche,39.91,-16.23,73.18,92,0,7.45,Boy's guest house
1,SA,Sakakah,40.21,29.97,78.80,21,0,8.05,محمصة الريف
2,SA,Riyadh,46.72,24.69,77.00,27,0,3.36,سلطان سليمان للتبريد والتكييف
3,US,Hays,-98.03,30.05,72.68,41,0,1.48,big d mini bucking bulls
4,IR,Sūsangerd,48.18,31.56,78.80,61,0,2.24,Saheb AL Zaman Mosque
5,IN,Bilhaur,80.08,26.85,75.87,45,0,4.52,* नेशनल ग्रामोफोन एजेंसी *
6,BR,Caravelas,-39.25,-17.71,74.50,84,0,7.49,Mateus' S/A
7,SA,Buraidah,43.98,26.33,73.40,35,0,5.82,ملاعب شعبية
8,IL,Kiryat Gat,34.76,31.61,75.99,85,0,2.46,שדרות גת 88 קרית גת
9,AR,San Juan,-68.54,-31.54,70.14,16,0,7.63,Larry's Un Sorbo Para La Distraccion


In [38]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_city_data.iterrows()]
marker_locations = ideal_city_data[["Latitude", "Longitude"]]

In [43]:
# Create a marker_layer using the Hotel Name, City and Country lists to fill the info box
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(humidity),
                                 point_radius = 4)
markers = gmaps.marker_layer(marker_locations, hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))